### Entrenando un algoritmo de machine learning

Para el presente entregable seleccioné un dataset obtenido del Instituto Nacional de Estadistica de Uruguay (INE).

El mismo corresponde a la encuesta continua de hogares realizada en el año 2017.

El objetivo del analisis sera trabajar con predicciones del ingreso en relación de dependencia de los encuestados.

Archivos utilizados:

P_2017_Terceros.sav -> el dataset que contiene los datos recopilados en la encuesta continua de hogares.
Dicccionario de Variables ECH 2017 -> funciona como anexo para determinar a que corresponde la codificación de cada columna del dataset.

In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [46]:
dfP2017 = pd.read_spss(r'..\\data\\P_2017_Terceros.sav')

In [47]:
columns = dfP2017.columns.values.tolist()
print(columns)

['numero', 'nper', 'anio', 'mes', 'dpto', 'nomdpto', 'secc', 'segm', 'loc_agr_13', 'nom_loc_agr_13', 'ccz', 'barrio', 'nombarrio', 'estred13', 'region_3', 'region_4', 'pesoano', 'pesosem', 'pesotri', 'pesomen', 'e557', 'e558', 'e26', 'e27', 'e29_1', 'e29_2', 'e29_3', 'e29_4', 'e29_5', 'e29_5_1', 'e29_6', 'e30', 'e31', 'e32', 'e33', 'e34', 'e35', 'e36', 'e185', 'e186_1', 'e186_2', 'e186_3', 'e186_4', 'e37', 'e37_2', 'e234_2', 'e38', 'e38_1', 'e39', 'e39_2', 'e235_2', 'e236', 'e236_2', 'e236_4', 'e45_1', 'e45_1_1', 'e45_1_1_1', 'e45_1_2', 'e45_1_2_1', 'e45_2', 'e45_2_1', 'e45_2_1_1', 'e45_2_2', 'e45_2_2_1', 'e45_3', 'e45_3_1', 'e45_3_1_1', 'e45_3_2', 'e45_3_2_1', 'e45_4', 'e45_4_1', 'e45_4_2', 'e45_4_3', 'e45_4_3_1', 'e45_5', 'e45_5_1', 'e45_5_1_1', 'e45_6', 'e45_7', 'e45_7_1', 'e237', 'e46', 'e47', 'e47_1', 'e190', 'e190_1', 'e190_1_1', 'e190_2', 'e190_2_1', 'e190_3', 'e190_3_1', 'e191', 'e192', 'e48', 'e49', 'e238', 'e239', 'e240_1', 'e240_2', 'e241', 'e242', 'e242_1', 'e193', 'e194', 

In [48]:
dfP2017.head(2)

,numero,nper,anio,mes,dpto,nomdpto,secc,segm,loc_agr_13,nom_loc_agr_13,...,indaceliac,indatuberc,indaoncolo,indasida,indaucc,PT1,PT2,PT4,pobre06,indigente06
0,2017000001,1.0,2017,Enero,Montevideo,MONTEVIDEO,17,006,01010,Montevideo,...,689.0,585.0,585.0,585.0,665.0,28779.0,28779.0,28779.0,0.0,0.0
1,2017000001,2.0,2017,Enero,Montevideo,MONTEVIDEO,17,006,01010,Montevideo,...,689.0,585.0,585.0,585.0,665.0,10100.0,0.0,0.0,0.0,0.0


A continuación se definen las variables a utilizar en el analisis y se genera un dataset independiente para realizar modificaciones sin afectar al dataset original

In [49]:
df_ysos = pd.DataFrame()

# variable de respuesta
df_ysos['ysos_dep'] = dfP2017['g126_1'] + dfP2017['g126_2'] + dfP2017['g126_3'] + dfP2017['g126_4'] \
                    + dfP2017['g134_1'] + dfP2017['g134_2'] + dfP2017['g134_3'] + dfP2017['g134_4']

# caracteristicas personales
df_ysos['edad'] = dfP2017['e27']
df_ysos['sexo'] = dfP2017['e26']
df_ysos['estrato'] = dfP2017['estred13']
df_ysos['ascendencia_negra'] = dfP2017['e29_1']
df_ysos['estado_civil'] = dfP2017['e36']
df_ysos['asiste_comedor']=dfP2017['e559']
df_ysos['seguro_privado']=dfP2017['e45_3']
df_ysos['pobre']=dfP2017['pobre06']
df_ysos['indigente']=dfP2017['indigente06']

## educación
df_ysos['anios_educ'] = dfP2017['e51_2'] + dfP2017['e51_3'] + dfP2017['e51_4'] + dfP2017['e51_5'] + dfP2017['e51_6'] \
                        + dfP2017['e51_7'] + dfP2017['e51_8'] + dfP2017['e51_9'] + dfP2017['e51_10'] + dfP2017['e51_11']

## relación de empleo
df_ysos['antiguedad'] = dfP2017['f88_2']
df_ysos['categoria_ocup'] = dfP2017['f73']
df_ysos['dimension_empresa']=dfP2017['f77']

df_ysos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118268 entries, 0 to 118267
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   ysos_dep           118268 non-null  float64 
 1   edad               118268 non-null  float64 
 2   sexo               118268 non-null  category
 3   estrato            118268 non-null  category
 4   ascendencia_negra  118268 non-null  category
 5   estado_civil       118268 non-null  category
 6   asiste_comedor     118268 non-null  category
 7   seguro_privado     118268 non-null  category
 8   pobre              118268 non-null  float64 
 9   indigente          118268 non-null  float64 
 10  anios_educ         118268 non-null  float64 
 11  antiguedad         118268 non-null  float64 
 12  categoria_ocup     118268 non-null  category
 13  dimension_empresa  118268 non-null  category
dtypes: category(8), float64(6)
memory usage: 6.3 MB


In [50]:
# genero archivo con datos a utilizar para alojar en Github, ya que el archivo .sav es muy pesado y no se puede guardar incluir al hacer commit a github.
export = '../data/ECH2017.xlsx'
df_ysos.to_excel(export, index=False)

In [51]:
df_ysos['dimension_empresa'].dtype

CategoricalDtype(categories=[                   0.0,  'De 10 a 19 personas',
                     'De 2 a 4 personas',  'De 20 a 49 personas',
                     'De 5 a 9 personas', 'De 50 personas o más',
                           'Una persona'],
, ordered=False)

In [52]:
df_ysos.head(2)

,ysos_dep,edad,sexo,estrato,ascendencia_negra,estado_civil,asiste_comedor,seguro_privado,pobre,indigente,anios_educ,antiguedad,categoria_ocup,dimension_empresa
0,18000.0,70.0,Hombre,Montevideo Bajo,No,Divorciado/a,No,No,0.0,0.0,6.0,7.0,Asalariado/a privado/a,De 10 a 19 personas
1,0.0,81.0,Mujer,Montevideo Bajo,No,Divorciado/a,No,No,0.0,0.0,4.0,0.0,0.0,0.0


In [53]:
#se eliminan del dataset todos los ingresos menores a 0
df_ysos = df_ysos[df_ysos['ysos_dep'] > 0]

In [54]:
df_ysos = df_ysos.loc[(df_ysos['categoria_ocup'] == "Asalariado/a privado/a") | (df_ysos['categoria_ocup'] == "Asalariado/a público/a")]

In [55]:
#transformación logaritmica de la variable ingresos
df_ysos['l_ysos_dep'] = np.log(df_ysos['ysos_dep'])
df_l_ysos = df_ysos.drop(columns=['ysos_dep'])
df_l_ysos.head(2)

,edad,sexo,estrato,ascendencia_negra,estado_civil,asiste_comedor,seguro_privado,pobre,indigente,anios_educ,antiguedad,categoria_ocup,dimension_empresa,l_ysos_dep
0,70.0,Hombre,Montevideo Bajo,No,Divorciado/a,No,No,0.0,0.0,6.0,7.0,Asalariado/a privado/a,De 10 a 19 personas,9.798127
2,52.0,Hombre,Costa Este,No,Casado/a (incluye separado/a y aún no se divor...,No,No,0.0,0.0,6.0,32.0,Asalariado/a público/a,De 50 personas o más,10.778956


A continuacón se realizan transformaciones a ciertas columnas para contar con la información agrupada de una forma que permita realizar analisis de forma más eficiente.

In [56]:
def map_estrato(estrato):
    if estrato in ['Montevideo Alto']:
        return 'Mdeo - Alto'
    elif estrato in ['Montevideo Medio', 'Montevideo Medio Bajo', 'Montevideo Medio Alto']:
        return 'Mdeo - Medio'
    elif estrato in ['Montevideo Bajo']:
        return 'Mdeo - Bajo'
    elif estrato in ['Zona Metropolitana']:
        return 'Metropolitana'
    elif estrato in ['Centro Norte', 'Centro Sur', 'Costa Este', 'Litoral Norte', 'Litoral Sur','Norte']:
        return 'Interior'

# se crea nueva columna que contenga los nombres de los grupos
df_l_ysos['estrato'] = df_l_ysos['estrato'].apply(map_estrato)

In [57]:
def map_dimenisonempresa(dimension):
    if dimension in ['Una persona', 'De 2 a 4 personas', 'De 5 a 9 personas', 'De 10 a 19 personas']:
        return 'Empresa_chica'
    elif dimension in ['De 20 a 49 personas']:
        return 'Empresa_mediana'
    elif dimension =='De 50 personas o más':
        return 'Empresa_grande'
    elif dimension ==0.0:
        return 'N/A'

# se crea nueva columna que contenga los nombres de los grupos
df_l_ysos['dimension_empresa'] = df_l_ysos['dimension_empresa'].apply(map_dimenisonempresa)

In [58]:
df_l_ysos.head(2)

,edad,sexo,estrato,ascendencia_negra,estado_civil,asiste_comedor,seguro_privado,pobre,indigente,anios_educ,antiguedad,categoria_ocup,dimension_empresa,l_ysos_dep
0,70.0,Hombre,Mdeo - Bajo,No,Divorciado/a,No,No,0.0,0.0,6.0,7.0,Asalariado/a privado/a,Empresa_chica,9.798127
2,52.0,Hombre,Interior,No,Casado/a (incluye separado/a y aún no se divor...,No,No,0.0,0.0,6.0,32.0,Asalariado/a público/a,Empresa_grande,10.778956


In [59]:
#Se agrupan las variables separandolas en categoricas y numericas.

categorical_columns = ['sexo', 'categoria_ocup','asiste_comedor','ascendencia_negra','estrato','seguro_privado', 'dimension_empresa']
numerical_columns = ['edad', 'anios_educ', 'antiguedad']

# Se genera la función transformadora
column_transform = make_column_transformer(
    (OneHotEncoder(drop='if_binary'), categorical_columns), # cuando son dos categorías, elimina una
    (StandardScaler(), numerical_columns), # comentado mientras no se decida estandarizar
    remainder='passthrough', # deja el resto de las columnas sin transformar
    verbose_feature_names_out=False)

In [60]:
df_l_ysos_transf = pd.DataFrame(
    column_transform.fit_transform(df_l_ysos), 
    columns=column_transform.get_feature_names_out()
)

df_l_ysos_transf.head()

,sexo_Mujer,categoria_ocup_Asalariado/a público/a,asiste_comedor_Sí,ascendencia_negra_Sí,estrato_Interior,estrato_Mdeo - Alto,estrato_Mdeo - Bajo,estrato_Mdeo - Medio,estrato_Metropolitana,seguro_privado_Sí,dimension_empresa_Empresa_chica,dimension_empresa_Empresa_grande,dimension_empresa_Empresa_mediana,edad,anios_educ,antiguedad,estado_civil,pobre,indigente,l_ysos_dep
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.29169,-1.207866,-0.164103,Divorciado/a,0.0,0.0,9.798127
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.896131,-1.207866,2.43851,Casado/a (incluye separado/a y aún no se divor...,0.0,0.0,10.778956
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.120821,0.966088,-0.892834,Divorciado/a,0.0,0.0,10.404263
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.887083,-0.773075,-0.78873,Soltero,0.0,0.0,7.783224
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.266835,-1.207866,-0.476416,Soltero,0.0,0.0,10.043249


In [61]:
predictoras = df_l_ysos_transf.drop('l_ysos_dep', axis=1)

In [62]:
df_l_ysos.head(2)

,edad,sexo,estrato,ascendencia_negra,estado_civil,asiste_comedor,seguro_privado,pobre,indigente,anios_educ,antiguedad,categoria_ocup,dimension_empresa,l_ysos_dep
0,70.0,Hombre,Mdeo - Bajo,No,Divorciado/a,No,No,0.0,0.0,6.0,7.0,Asalariado/a privado/a,Empresa_chica,9.798127
2,52.0,Hombre,Interior,No,Casado/a (incluye separado/a y aún no se divor...,No,No,0.0,0.0,6.0,32.0,Asalariado/a público/a,Empresa_grande,10.778956


In [63]:
X_train, X_test, y_train, y_test = train_test_split(predictoras,
                                                    df_l_ysos_transf['l_ysos_dep'], 
                                                    test_size=0.20, 
                                                    random_state=1234)

#### Regresión Lineal

A continuación se plantea el modelo propiamente dicho, para una primera instancia se trabaja solamente con la variable que toma los años de educación del encuestado.

In [64]:
reg = linear_model.LinearRegression()

In [65]:
modelo_X_train = X_train[['anios_educ']]
modelo_X_test = X_test[['anios_educ']]

modelo = reg.fit(modelo_X_train, y_train)

In [66]:
# predicción en train
modelo_y_train_pred = modelo.predict(modelo_X_train)

# predicción en test
modelo_y_test_pred = modelo.predict(modelo_X_test)

In [67]:
modelo_mse_train = mean_squared_error(y_train, modelo_y_train_pred)
print(f"Error cuadratico medio del modelo en train: ", modelo_mse_train)

Error cuadratico medio del modelo en train:  0.4840089924851105


In [68]:
modelo_mse_test = mean_squared_error(y_test, modelo_y_test_pred)
print(f"Error cuadratico medio del modelo en test: ", modelo_mse_test)

Error cuadratico medio del modelo en test:  0.4834283901748652
